# Preprocess subject data

In [5]:
import sys
import os
import logging
import ipyparallel as ipp

# Set up the source path
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)

# Import necessary modules from src
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils
from src.preprocessing.classes import Subject

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load in subject IDs and paths from configuration
config = Config.from_json(configs.CFGLog)

# Define subject IDs
subject_ids = {
    "chronic_low_back_pain": config.data.chronic_low_back_pain.subject_ids.CP + config.data.chronic_low_back_pain.subject_ids.HC,
    # "chronic_pancreatitis": config.data.chronic_pancreatitis.subject_ids.CP,
}

# CLBP processed path
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
PERISTIM_TIME_WIN = 5
times_tup, time_win_path = pre_utils.get_time_window(PERISTIM_TIME_WIN)
time_range = (times_tup[0], times_tup[-1])

# Log total subjects
total_subjects = sum(len(ids) for ids in subject_ids.values())
logger.info(f"Total subjects: {total_subjects}")
logger.info(subject_ids)

# Function to preprocess each subject
def preprocess_subject(sub_id, group, clbp_processed_path, time_range, PERISTIM_TIME_WIN):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)

    try:
        preprocessed_data_path = clbp_processed_path if group == "chronic_low_back_pain" else None
        subject = Subject(sub_id, group, preprocessed_data_path=preprocessed_data_path)
        logger.info(f"Processing subject: {sub_id} in group: {group}")
        # subject.preprocess()
        # subject.get_cleaned_epochs(time_range, PERISTIM_TIME_WIN)

        # Load in Raw data
        subject.load_preprocessed()
        subject.save(subject.preprocessed_raw, "preprocessed_raw", as_vhdr=True)
        
        # Load in Epochs data
        subject.load_epochs()
        subject.save(subject.epochs.data, "epochs", as_mat=True)
        
        logger.info(f"Completed processing subject: {sub_id} in group: {group}")
    except FileNotFoundError as e:
        logger.error(f"FileNotFoundError for subject: {sub_id} - {e}")
    except Exception as e:
        logger.error(f"An error occurred for subject: {sub_id} - {e}")

# Connect to IPyParallel cluster
rc = ipp.Client(profile='preprocessing')
dview = rc[:]

# Push the source path and necessary imports to all engines
dview.execute(f"import sys")
dview.execute(f"sys.path.append('{src_path}')")

# Sync necessary imports on all engines
with dview.sync_imports():
    from src.utils.config import Config
    import logging

# Push necessary configurations and functions to all engines
dview.push({
    'src_path': src_path,
    'config': config,
    'clbp_processed_path': clbp_processed_path,
    'time_range': time_range,
    'PERISTIM_TIME_WIN': PERISTIM_TIME_WIN,
    'preprocess_subject': preprocess_subject
})

# Map the function to the subjects in parallel
results = []
for group in subject_ids:
    results += dview.map(
        preprocess_subject, 
        subject_ids[group], 
        [group] * len(subject_ids[group]), 
        [clbp_processed_path] * len(subject_ids[group]), 
        [time_range] * len(subject_ids[group]), 
        [PERISTIM_TIME_WIN] * len(subject_ids[group])
    )

# Wait for results to complete
rc.wait_interactive()


2024-07-16 14:41:08,253 - __main__ - INFO - Total subjects: 43
2024-07-16 14:41:08,255 - __main__ - INFO - {'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}


[-2.5,0.0,2.5]
importing Config from src.utils.config on engine(s)
importing logging on engine(s)


unknown:   0%|          | 0/7 [00:00<?, ?tasks/s]

In [ ]:
# inspect log
logger.info(results)

2024-07-15 16:25:11,290 - __main__ - INFO - [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        subject.preprocess()
        if not subject.file_exists("preprocessed_raw", "vhdr"):
            subject.save(subject.preprocessed_raw, "preprocessed_raw", as_vhdr=True)
        
        # Load in Epochs data
        subject.load_epochs()
        if not subject.file_exists("epochs", "mat"):
            subject.save(subject.epochs.data, "epochs", as_mat=True)

imported src module
imported preprocessing module
imported configs module
imported utils module
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}
Loaded preprocessed entire for subject 018
Loaded epochs for subject 018
Loaded preprocessed entire for subject 022
Loaded epochs for subject 022
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/022_epochs.
Loaded preprocessed entire for subject 024
Loaded epochs for subject 024
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/024_epochs.
Loaded preprocessed entire for subject 031
Loaded epochs for subject 031
Saved ep

/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:202: RuntimeWarning: Encountered data in 'int' format. Converting to float32.
  data_object.export(save_file_path)
/home/wanglab/mambaforge/envs/highpaineeg/lib/python3.11/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/044_preprocessed_raw.vhdr.
Loaded epochs for subject 044
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/044_epochs.
Loaded preprocessed entire for subject 048


/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:202: RuntimeWarning: Encountered data in 'int' format. Converting to float32.
  data_object.export(save_file_path)


Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/048_preprocessed_raw.vhdr.
Loaded epochs for subject 048
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/048_epochs.
Loaded preprocessed entire for subject 049


...

Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C6._preprocessed_raw.vhdr.
Loaded epochs for subject C6.
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C6._epochs.
Loaded preprocessed entire for subject C7.


/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:202: RuntimeWarning: Encountered data in 'int' format. Converting to float32.
  data_object.export(save_file_path)


Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C7._preprocessed_raw.vhdr.
Loaded epochs for subject C7.
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C7._epochs.
Loaded preprocessed entire for subject C9.


/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:202: RuntimeWarning: Encountered data in 'int' format. Converting to float32.
  data_object.export(save_file_path)


Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C9._preprocessed_raw.vhdr.
Loaded epochs for subject C9.
Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/C9._epochs.


In [ ]:
subject.epochs

Number of events,73
Events,Back HS: 17Back NS: 17Hand HS: 20Hand NS: 19
Time range,-2.500 – 2.502 s
Baseline,-2.500 – 0.000 s
